<a href="https://colab.research.google.com/github/ZainAli24/Quater_04/blob/main/OpenAi_AgentsSDK_Quiz_Prepreation_onsite01_Structured_Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Q.1 LLM structured_output ke mutabik answer na kare Toh kia loop continue rahe ga ya phir error raise karega?**

Aapka sawal yeh hai ke agar LLM structured output ke mutabik jawab nahi deta, yani `AgentOutputSchema` ke defined schema ke hisaab se output mein koi attribute miss hota hai ya extra attribute aata hai, toh OpenAI Agents SDK ki official documentation ke mutabik agent loop continue rahega ya ruk jayega. Diye gaye link ([https://openai.github.io/openai-agents-python/agents/#output-types](https://openai.github.io/openai-agents-python/agents/#output-types)) aur related documentation ka analysis karke jawab deta hoon.

### Documentation ka Analysis
Documentation ke section “Output types” mein `output_type` parameter aur structured outputs ke behavior ka zikr hai. Is section aur related pages (jaise [Running agents](https://openai.github.io/openai-agents-python/running/)) se relevant details yeh hain:

1. **Structured Outputs ka Behavior:**
   - Documentation ke mutabik, jab `output_type` set hota hai (jaise Pydantic model, dataclass, ya TypedDict), toh agent loop tab tak chalta hai jab tak LLM is `output_type` ke mutabik structured output na produce kar de.[](https://openai.github.io/openai-agents-python/agents/)
   - Agar LLM structured output produce karta hai, toh yeh `AgentOutputSchema.validate_json` method ke zariye validate hota hai. Agar validation fail hoti hai (jaise missing attribute ya extra attribute ke wajah se), toh ek `ModelBehaviorError` raise hota hai.[](https://openai.github.io/openai-agents-python/running_agents/)

2. **Agent Loop ka Flow:**
   - [Running agents](https://openai.github.io/openai-agents-python/running/) section mein agent loop ka flow describe kiya gaya hai:
     - Agent input ke saath invoke hota hai.
     - Agar final output (matching `output_type`) milta hai, loop terminate hota hai.
     - Agar handoff hota hai, toh naya agent chalta hai.
     - Agar tool calls hote hain, toh unke results ke saath loop dobara chalta hai.
     - Agar `max_turns` exceed ho jata hai ya guardrail tripwire trigger hota hai, toh exception raise hoti hai (`MaxTurnsExceeded` ya `GuardrailTripwireTriggered`).[](https://openai.github.io/openai-agents-python/ref/run/)
   - Documentation mein yeh bhi mention hai ke `ModelBehaviorError` raise hota hai jab model invalid output deta hai, jaise malformed JSON ya schema ke mutabik nahi. Yeh error `AgentsException` ka subclass hai.[](https://openai.github.io/openai-agents-python/running_agents/)

3. **Error Handling aur Loop Termination:**
   - Documentation ke [Running agents](https://openai.github.io/openai-agents-python/running/) section mein exceptions ka zikr hai:
     - `ModelBehaviorError` jab raise hota hai, toh yeh agent run ko interrupt karta hai, aur loop terminate ho jata hai.[](https://openai.github.io/openai-agents-python/running_agents/)
     - Code ke `Runner.run` method (jo pehle ke jawab mein analyze kiya gaya) mein `try-except` block `AgentsException` (jisme `ModelBehaviorError` shamil hai) ko catch karta hai aur exception ko re-raise karta hai, jo loop ko rok deta hai.
   - Documentation mein koi aisa mechanism nahi mention kiya gaya jo `ModelBehaviorError` ke baad loop ko continue rakhe, jaise retry logic ya fallback output.

4. **Related Context:**
   - [Results](https://openai.github.io/openai-agents-python/results/) section mein bataya gaya hai ke `final_output` tab produce hota hai jab agent `output_type` ke mutabik output deta hai. Agar output invalid hai, toh loop aage nahi badhta aur error throw hota hai.[](https://openai.github.io/openai-agents-python/results/)
   - [Streaming](https://openai.github.io/openai-agents-python/streaming/) section mein bhi similar behavior hai, jahan `ModelBehaviorError` loop ko terminate karta hai aur `RunResultStreaming.is_complete` set ho jata hai.[](https://openai.github.io/openai-agents-python/streaming/)

### Documentation se Conclusion
Official documentation ke mutabik, agar LLM structured output ke mutabik jawab nahi deta (jaise `AgentOutputSchema` mein defined schema ke hisaab se attribute miss hota hai ya extra attribute aata hai), toh:
- `AgentOutputSchema.validate_json` method ek `ModelBehaviorError` raise karega.
- Yeh error `AgentsException` ke roop mein catch hota hai, aur agent loop **ruk jayega** kyunki exception re-raised hoti hai aur koi retry ya continue logic documentation mein mention nahi hai.

**Final Jawab:** Documentation ke hisaab se, agent loop **nahi continue rahega** aur **ruk jayega** agar LLM structured output ke mutabik jawab nahi deta.[](https://openai.github.io/openai-agents-python/running_agents/)

### Additional Notes
- Documentation mein aisa koi feature ya configuration nahi bataya gaya jo invalid structured output ke case mein loop ko automatically continue rakhe. Agar aapko loop ko continue rakhna hai, toh aapko SDK ke bahar custom error handling implement karna hoga, jaise:
  ```python
  try:
      result = await Runner.run(agent, input)
  except ModelBehaviorError as e:
      print(f"Invalid output: {e}")
      # Retry ya fallback logic
  ```
--------

# **2) _strict_json_schema checking behaviour:**

Aapka sawal yeh hai ke OpenAI Agents SDK mein kaunsa method ya attribute `strict` output type ko check karta hai, aur agar isko `False` rakha jaye toh kya model ka response thoda idhar udhar ho sakta hai. Saath hi, agar is case mein model `StructuredOutput` (yaani `TranslationOutput` schema) ke mutabik jawab na de, toh kya error raise hoga ya nahi. Main isko code aur OpenAI Agents SDK ke context mein analyze karta hoon, khas tor pe aapke diye gaye code (`main.py`) aur pehle ke discussions ke reference se.

### Analysis: Strict Output Type Check
OpenAI Agents SDK mein structured output ka validation `AgentOutputSchema` class ke zariye hota hai, jo `output_type` parameter ke basis par JSON schema generate aur validate karti hai. Aapke code mein `TranslationOutput` class (`original_text`, `translated_text`, `token_id`) output schema define karti hai.

#### Relevant Attribute: `strict_json_schema`
- **Location:** `AgentOutputSchema` class mein `_strict_json_schema` attribute hai jo control karta hai ke JSON schema strict mode mein hai ya nahi.
  ```python
  _strict_json_schema: bool
  """Whether the JSON schema is in strict mode. We **strongly** recommend setting this to True,
  as it increases the likelihood of correct JSON input."""
  ```
- **Default Value:** `_strict_json_schema` ka default value `True` hai jab `AgentOutputSchema` initialize hoti hai:
  ```python
  def __init__(self, output_type: type[Any], strict_json_schema: bool = True):
      self._strict_json_schema = strict_json_schema
  ```
- **Behavior:**
  - Jab `_strict_json_schema=True` hota hai:
    - Schema strictly enforce hota hai, yani model ka output exact schema ke mutabik hona chahiye.
    - Required fields missing hona ya extra fields (jo schema mein nahi hain) shamil hona validation failure ka sabab banta hai.
    - `ensure_strict_json_schema` function schema mein `additionalProperties: False` set karta hai, jo extra fields ko disallow karta hai.
  - Jab `_strict_json_schema=False` hota hai:
    - Schema ka enforcement thoda relaxed hota hai. Extra fields allowed ho sakte hain, lekin required fields abhi bhi zaroori hain.
    - Model ka response thoda "idhar udhar" ho sakta hai, yani schema se thodi deviation possible hai (jaise extra attributes).

#### Method Jo Strictness Check Karta Hai
- **Method:** `validate_json` method in `AgentOutputSchema` class output ko schema ke against validate karta hai:
  ```python
  def validate_json(self, json_str: str) -> Any:
      validated = _json.validate_json(json_str, self._type_adapter, partial=False)
      # ... Additional checks for wrapped outputs ...
      return validated
  ```
  - Yeh method Pydantic ke `TypeAdapter` ka use karta hai aur `partial=False` ke saath strict validation karta hai.
  - Agar `_strict_json_schema=True` hai, toh schema ke strict rules (jaise `additionalProperties: False`) apply hote hain.
  - Agar `_strict_json_schema=False` hai, toh validation thodi lenient hoti hai, lekin required fields ka check abhi bhi hota hai.

- **Supporting Function:** `ensure_strict_json_schema` function strict mode ke liye schema ko modify karta hai jab `_strict_json_schema=True` hota hai:
  ```python
  if self._strict_json_schema:
      try:
          self._output_schema = ensure_strict_json_schema(self._output_schema)
      except UserError as e:
          raise UserError(...) from e
  ```

### `_strict_json_schema=False` Ka Asar
Agar aap `_strict_json_schema=False` set karte hain, toh:
- **Extra Fields Allowed:** Model agar extra attributes (jo schema mein nahi hain, jaise `target_language`) return karta hai, toh validation fail nahi hogi, jab tak required fields (`original_text`, `translated_text`, `token_id`) maujood hain.
- **Missing Required Fields:** Agar required fields missing hote hain, toh validation abhi bhi fail hogi, kyunki Pydantic ka default behavior required fields ko enforce karta hai.
- **Model Response Flexibility:** Model ka response thoda flexible ho sakta hai, yani instructions ke mutabik agar model schema se thodi deviation karta hai (jaise extra fields ya alag structure), toh validation pass ho sakti hai agar required fields present hain.

#### Code Modification
Aapke code mein `_strict_json_schema=False` set karne ke liye, `Agent` ke initialization mein `output_type` ke saath `AgentOutputSchema` explicitly pass karna hoga, kyunki `Agent` by default `AgentOutputSchema(output_type, strict_json_schema=True)` banata hai.

**Modified Code:**
```python
from agents import Agent, Runner, OpenAIChatCompletionsModel, set_tracing_disabled, AsyncOpenAI, AgentOutputSchema
from pydantic import BaseModel
from dotenv import load_dotenv
import os
import asyncio
import logging

# Logging setup
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

load_dotenv()

OPENAI_AGENTS_DISABLE_TRACING = os.getenv("OPENAI_AGENTS_DISABLE_TRACING")
API_KEY = os.getenv("GEMINI_API_KEY")
BASE_URL = os.getenv("BASE_URL")

set_tracing_disabled(OPENAI_AGENTS_DISABLE_TRACING)

external_client = AsyncOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

class TranslationOutput(BaseModel):
    original_text: str
    translated_text: str
    token_id: str | int

agent = Agent(
    name="Translator",
    instructions="""You are a translation assistant. Translate the input text to German and return a JSON object with the following fields: translated_text, target_language. Do not include original_text or token_id.""",
    model=model,
    output_type=AgentOutputSchema(TranslationOutput, strict_json_schema=False)  # Strict mode disabled
)

async def main():
    try:
        logger.debug("Starting agent run...")
        result = await Runner.run(agent, "Hi, My Name is Zain and What's your good name?")
        logger.debug(f"Run result: {result}")
        print(result.final_output)
    except Exception as e:
        logger.error(f"Error occurred: {e}")
        print(f"Error: {e}")

if __name__ == "__main__":
    asyncio.run(main())
```

**Changes:**
- `output_type=AgentOutputSchema(TranslationOutput, strict_json_schema=False)` set kiya gaya hai, jo strict JSON schema validation ko disable karta hai.
- Instructions ko modify kiya gaya hai taki model `translated_text` aur `target_language` return kare, jabke `original_text` aur `token_id` (jo required hain) na de. Yeh schema mismatch create karega.
- Logging add kiya gaya hai for debugging.

### Case: Model Structured Output ke Mutabik Jawab Nahi Deta
Ab dekhte hain ke `_strict_json_schema=False` ke saath agar model `TranslationOutput` ke mutabik jawab nahi deta, toh kya hota hai.

#### Scenario 1: Model Missing Required Fields Deta Hai
**Instructions Output Example:**
Agar model instructions ke mutabik yeh output deta hai:
```json
{
  "translated_text": "Hallo, mein Name ist Zain, und wie ist Ihr Name?",
  "target_language": "German"
}
```

**Behavior:**
- `TranslationOutput` schema mein `original_text` aur `token_id` required hain, lekin yeh output mein missing hain.
- `_strict_json_schema=False` hone ke bawajood, Pydantic ka `TypeAdapter` required fields ke liye validation karta hai (`partial=False` in `validate_json`).
- Validation fail hogi kyunki required fields missing hain.
- **Error Raise Hoga:** `ModelBehaviorError` raise hoga, jo `AgentsException` ka subclass hai. Error message kuch aisa hoga:
  ```
  Error: Missing required fields: original_text, token_id
  ```
- **Agent Loop Behavior:** Jaise pehle ke jawab mein bataya gaya, `ModelBehaviorError` ke raise hone par `Runner.run` method loop ko terminate kar deta hai, aur error `main` function ke `except` block mein catch hota hai.

#### Scenario 2: Model Extra Fields Deta Hai
**Instructions Output Example:**
Agar model yeh output deta hai:
```json
{
  "translated_text": "Hallo, mein Name ist Zain, und wie ist Ihr Name?",
  "original_text": "Hi, My Name is Zain and What's your good name?",
  "token_id": "greeting_001",
  "target_language": "German",
  "extra_info": "Translation completed"
}
```

**Behavior:**
- Yeh output `TranslationOutput` ke required fields (`original_text`, `translated_text`, `token_id`) ko satisfy karta hai.
- `_strict_json_schema=False` hone ke wajah se extra fields (`target_language`, `extra_info`) allowed hain, kyunki `additionalProperties: False` enforce nahi hota.
- Validation **pass** ho jayegi, aur koi error raise nahi hoga.
- **Agent Loop Behavior:** Loop terminate ho jayega aur `final_output` return ho jayega, jo schema ke mutabik hai lekin extra fields ke saath.
- **Output Example:**
  ```
  original_text="Hi, My Name is Zain and What's your good name?"
  translated_text="Hallo, mein Name ist Zain, und wie ist Ihr Name?"
  token_id="greeting_001"
  target_language="German"
  extra_info="Translation completed"
  ```

#### Scenario 3: Model Plain Text Deta Hai
**Instructions Change:**
Agar instructions aise hain:
```python
instructions="""You are a translation assistant. Return ONLY the translated text in German as plain text. Do not return JSON or any structured output."""
```

**Output Example:**
```
Hallo, mein Name ist Zain, und wie ist Ihr Name?
```

**Behavior:**
- `_strict_json_schema=False` hone ke bawajood, `AgentOutputSchema` expect karta hai ke output valid JSON ho, kyunki `output_type=TranslationOutput` ek structured schema hai.
- Plain text JSON nahi hai, isliye `validate_json` method fail ho jayega.
- **Error Raise Hoga:** `ModelBehaviorError` raise hoga, message kuch aisa hoga:
  ```
  Error: Expected a valid JSON object matching the schema, got 'Hallo, mein Name ist Zain, und wie ist Ihr Name?'
  ```
- **Agent Loop Behavior:** Loop ruk jayega, aur error `main` function ke `except` block mein catch ho jayega.

### Conclusion
- **Kon Si Method/Attribute Strictness Check Karta Hai?**
  - `_strict_json_schema` attribute in `AgentOutputSchema` strict mode ko control karta hai.
  - `validate_json` method validation perform karta hai, aur `ensure_strict_json_schema` function strict rules apply karta hai jab `_strict_json_schema=True` hota hai.
- **_strict_json_schema=False Ka Asar:**
  - Extra fields allowed ho jate hain, lekin required fields abhi bhi zaroori hain.
  - Model ka response thoda flexible ho sakta hai (extra attributes shamil ho sakte hain).
- **Agar Model Structured Output ke Mutabik Jawab Nahi Deta:**
  - **Missing Required Fields ya Plain Text:** `ModelBehaviorError` raise hoga, aur agent loop **ruk jayega**.
  - **Extra Fields:** Agar required fields present hain, toh validation pass ho jayegi, koi error nahi hoga, aur loop terminate ho jayega valid output ke saath.
- **Error Raise Hoga Ya Nahi:**
  - Error tab raise hoga jab required fields missing hon ya output JSON na ho (jaise plain text).
  - Error nahi hoga agar required fields present hon aur extra fields shamil hon.

### Recommendation
- Aapke code mein `_strict_json_schema=False` set karne ke saath instructions aise rakhein jo required fields (`original_text`, `token_id`) ko exclude karein, jaise:
  ```python
  instructions="""You are a translation assistant. Return a JSON object with translated_text and target_language. Do not include original_text or token_id."""
  ```
  Isse validation fail hogi aur `ModelBehaviorError` raise hoga.
- Debugging ke liye logging ka use continue rakhein taki model ka raw output track ho sake.
- Agar model abhi bhi schema ke mutabik output deta hai, toh Gemini API ke behavior ya `model_settings` check karein, ya `output_type=str` try karein (jaise pehle suggest kiya gaya).

---------

#  **3. Output Type Kaise Kaam Karta Hai?**
## **Backend Working:**

- ### output_type=MathHomeworkOutput Pydantic model ka schema banata hai, jo OpenAI Agents SDK ke backend mein JSON schema mein convert hota hai.
- ### Yeh JSON schema LLM ko as a tool schema diya jata hai. OpenAI ke modern models (jaise GPT-4) structured output ke liye JSON schema ko samajh sakte hain.
- ### LLM is schema ke mutabiq output generate karta hai, aur SDK is output ko Pydantic model (MathHomeworkOutput) mein parse karta hai.
- ### Pydantic phir validate karta hai ke output sahi format mein hai ya nahi. Agar format galat hai, toh error raise hota hai.

In [1]:
!pip install -qU openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.3/734.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from pydantic import BaseModel
from agents import Agent,Runner,OpenAIChatCompletionsModel,AsyncOpenAI,function_tool, set_tracing_disabled
from google.colab import userdata

In [4]:
gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

In [5]:
from agents import enable_verbose_stdout_logging

enable_verbose_stdout_logging()

set_tracing_disabled(True)

In [ ]:
from agents import Agent, Runner, OpenAIChatCompletionsModel, set_tracing_disabled, AsyncOpenAI, function_tool
from pydantic import BaseModel
from dotenv import load_dotenv
import os
import asyncio
import requests

load_dotenv()

OPENAI_AGENTS_DISABLE_TRACING = os.getenv("OPENAI_AGENTS_DISABLE_TRACING")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
BASE_URL = os.getenv("BASE_URL")
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")

if not OPENWEATHER_API_KEY:
    raise ValueError("OPENWEATHER_API_KEY not found in .env file")

set_tracing_disabled(OPENAI_AGENTS_DISABLE_TRACING)

external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url=BASE_URL,
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

class WeatherOutput(BaseModel):
    location: str
    temperature: float
    description: str  # Added to match instructions and tool output
    humidity: int
    pressure: int

@function_tool
def fetch_weather(city_name: str) -> dict:
    print(f"Fetching weather for: for {city_name}")  # Debug to confirm tool call
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = f"{base_url}q={city_name}&appid={OPENWEATHER_API_KEY}&units=metric"

    try:
        response = requests.get(complete_url)
        response.raise_for_status()
        data = response.json()

        if data.get("cod") != 200:
            raise Exception(data.get("message", "Unable to fetch weather data"))

        return {
            "location": data["name"],
            "temperature": data["main"]["temp"],
            "description": data["weather"][0]["description"],
            "humidity": data["main"]["humidity"],
            "pressure": data["main"]["pressure"]
        }

    except Exception as e:
        print(f"Error in fetch_weather: {e}")  # Debug
        raise Exception(f"Failed to fetch weather for {city_name}: {str(e)}")

agent = Agent(
    name="WeatherAssistant",
    instructions="""You are a a weather assistant. Use the fetch_weather tool to get real-time weather data for the specified city. Return the response in the WeatherOutput schema format, including location, temperature, description, humidity, and pressure.""",
    model=model,
    output_type=WeatherOutput,
    tools=[fetch_weather]
)

async def main():
    try:
        result = await Runner.run(agent, input="What's the weather in Gujar Khan?")  # Fixed query with city name
        print(result.final_output)
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    asyncio.run(main())

